In [1]:
# Importing Libraries
import requests
import json
import pandas as pd
import os

In [2]:
# Loading CityBikes data from Part 1
ReddingBikes = pd.read_csv('../data/ReddingBikesDF.csv')

# [Foursquare](https://docs.foursquare.com/developer/reference/places-api-overview)

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# Call Foursqaure API function
def getVenuesFS(latitude, longitude, radius, api_key, categories, fields, limit):
    """
    Get venues from foursquare with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : Foursquare category codes. Separate multiple ids with commas
        fields (str): Foursquare fields. Seperate multiple fields with commas.
        limit (int): Number of results to return, max 50

    Returns:
        response: response object from the requests library.
    """
    headers = {
        "accept": "application/json",
        "Authorization": api_key
    }

    ll=str(latitude) + ',' + str(longitude)

    url = f'https://api.foursquare.com/v3/places/search?ll={ll}&radius={radius}&categories={categories}&limit={str(limit)}&fields={fields}'

    results = requests.get(url, headers=headers)

    return results

Foursquare [category codes](https://docs.foursquare.com/data-products/docs/categories) for Dining & Drinking and POIs
* 10000	Arts and Entertainment
* 13000   Dining & Drinking
* 16000	Landmarks and Outdoors
* 18000	Sports and Recreation


In [4]:
# Setting Constant Variables 
FOURSQUARE_KEY = os.getenv('Foursquare_API_KEY')
radius = 1000
categories = '10000,13000,16000,18000'
fields = 'fsq_id,name,location,categories,distance,verified,rating,stats,popularity,price'
limit = 50

In [5]:
# Pulling all info for each bike station into a list
FoursquareVenues = [] 

for i in range(len(ReddingBikes)):
    venues = getVenuesFS(ReddingBikes['latitude'][i], ReddingBikes['longitude'][i], radius, FOURSQUARE_KEY, categories, fields, limit).json()
    FoursquareVenues.append(venues)    


In [6]:
# Function to return category group using category id code
def CategoryGroup(categoryID):
    """
    Returns Foursquare Category Group from CategoryID 

    Args:
        categoryID (str): String of category id code from foursqaure

    Returns:
        Category Group from foursquare codes
    """
    if categoryID.startswith('10'): 
        return 'Arts and Entertainment'
    elif categoryID.startswith('11'): 
        return 'Business and Professional Services'
    elif categoryID.startswith('12'): 
        return 'Community and Government'
    elif categoryID.startswith('13'): 
        return 'Dining and Drinking'
    elif categoryID.startswith('14'): 
        return 'Event'
    elif categoryID.startswith('15'): 
        return 'Health and Medicine'
    elif categoryID.startswith('16'):
        return 'Landmarks and Outdoors'
    elif categoryID.startswith('17'): 
        return 'Retail'
    elif categoryID.startswith('18'): 
        return 'Sports and Recreation'
    elif categoryID.startswith('19'): 
        return 'Travel and Transportation'



### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
# Function to return list of desired results from each Foursquare venue
def ParseFoursqaureResults(venue): 
    """ 
    Parses elements from a venue in the list of results given by a Foursquare Query.

    Args: 
        venue (dict): A venue from the list of Foursquare Venues for each station

    Returns a list of preset elements in the following order from the venue.  
    List order: ['fsqID', 'venueName', 'formattedAddress', 'catID', 'catName', 'catIDGroup', 'distance', 'rating', 'totalRatings', 'totalPhotos', 'totalTips', 'popularity', 'price', 'storeVerified']
    """
    # FS core data. All venues have this information 
    fsqID = venue['fsq_id']
    venueName = venue['name']
    formattedAddress = venue['location']['formatted_address']
    catID = venue['categories'][0]['id']
    catName = venue['categories'][0]['name']
    catIDGroup = CategoryGroup(str(catID))
    distance = venue['distance']
    
    # FS Rich data. Fields which may not exist
    try:
        rating = venue['rating']
    except: 
        rating = None

    try:
        totalRatings = venue['stats']['total_ratings']
        totalPhotos = venue['stats']['total_photos']
        totalTips = venue['stats']['total_tips']
    except: 
        totalRatings = None
        totalPhotos = None
        totalTips = None

    try:
        popularity = venue['popularity']
    except: 
        popularity = None

    try:
        price = venue['price']
    except: 
        price = None

    try:
        storeVerified = venue['verified']    
    except: 
        storeVerified = None

    returnList = [fsqID, venueName, formattedAddress, catID, catName, catIDGroup, distance, rating, totalRatings, totalPhotos, totalTips, popularity, price, storeVerified]

    return returnList

In [8]:
# Parsing all results into a list using function for tidy data that can be converted to DF
fsVenuesList = []

for i in range(len(FoursquareVenues)):
    for venues in FoursquareVenues[i]['results']:
        returnList = ParseFoursqaureResults(venues)
        returnList.append(i)
        fsVenuesList.append(returnList)

### Put your parsed results into a DataFrame

In [9]:
fsColumnsHeader = ['fsqID', 'venueName', 'formattedAddress', 'catID', 'catName', 'catIDGroup', 'distance', 'rating', 'totalRatings', 'totalPhotos', 'totalTips', 'popularity', 'price', 'storeVerified','stationIndex']

FSVenuesDF = pd.DataFrame(fsVenuesList)
FSVenuesDF.columns=fsColumnsHeader

In [10]:
FSVenuesDF.head()

,fsqID,venueName,formattedAddress,catID,catName,catIDGroup,distance,rating,totalRatings,totalPhotos,totalTips,popularity,price,storeVerified,stationIndex
0,4b5138a2f964a520cc4727e3,Market Street Steakhouse,"1777 Market St, Redding, CA 96001",13383,Steakhouse,Dining and Drinking,188,8.7,14.0,12.0,6.0,0.966268,3.0,False,0
1,581ccd7f0d63b178d22e9e54,Theory Collaborative,"1250 California St, Redding, CA 96001",13034,Café,Dining and Drinking,818,8.9,35.0,34.0,4.0,0.978369,1.0,True,0
2,5a110c7f0fe7a07167d2f62f,Taste & See Creamery,"1419 Market St, Redding, CA 96001",13046,Ice Cream Parlor,Dining and Drinking,579,8.0,9.0,11.0,1.0,0.974588,1.0,False,0
3,4c4bf3589c8d2d7f72f4b76b,Club 501,"1244 California St, Redding, CA 96001",13011,Gay Bar,Dining and Drinking,830,8.3,12.0,6.0,4.0,0.905461,2.0,False,0
4,4bfd76e4bf6576b08551adb8,Wienerschnitzel,"1120 Market St, Redding, CA 96001",13058,Hot Dog Joint,Dining and Drinking,972,7.6,12.0,44.0,1.0,0.987294,1.0,False,0


In [11]:
# FSVenuesDF.to_csv('../data/FoursquareVenuesDF.csv', sep=',')

# [Yelp API](https://docs.developer.yelp.com/reference/v3_business_search)

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [74]:
# Call Yelp API function
def getVenuesYelp(latitude, longitude, radius, YelpKey, categories, limit):
    """
    Get venues from yelp with a specified place type and coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
        api_key (str): foursquare API to use for query
        categories (str) : string of category types. Seperate multiple types with commas
        limit (int): Number of results to return, max 50

    Returns:
        response: response object from the requests library.
    """
    headers = {
        "accept": "application/json",
        "Authorization": rf'Bearer {YelpKey}'
    }

    url = fr"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}&categories={categories}&sort_by=distance&limit={limit}"

    results = requests.get(url, headers=headers)

    return results

[Yelp Categories List](https://docs.developer.yelp.com/docs/resources-categories) for dining and POIs, selected to match Foursquare Categories

* Active Life (active)
* Arts & Entertainment (arts)
* Food (food)
* Restaurants (restaurants)
* Bars (bars)
* Landmarks & Historical Buildings (landmarks)

Event Categories
* Music (music)
* Visual Arts (visual-arts)
* Performing Arts (performing-arts)
* Film (film)
* Lectures & Books (lectures-books)
* Food & Drink (food-and-drink)
* Sports & Active Life (sports-active-life)
* Nightlife (nightlife)


In [75]:
# Setting Constant Variables 
YelpKey = os.getenv('YelpAPIKey')
radius = 1000
categories = 'active,arts,food,restauraunt,bars,landmarks'
limit = 50

In [99]:
# Pulling all info for each bike station into a list
YelpVenues = [] 

# Category list created as the results from calling the API does not return the highest level of category Grouping.
# Thus to create a combined dataframe of all the POIs and still be able to differentiate the different groups, seperate calls were needed.
# categories = 'food-and-drink'

for i in range(len(ReddingBikes)):
    venues = getVenuesYelp(ReddingBikes['latitude'][i], ReddingBikes['longitude'][i], radius, YelpKey, categories, limit).json()
    YelpVenues.append(venues)    

# categories = 'food-and-drink' 
# for i in range(len(ReddingBikes)):
#     venues = getVenuesYelp(ReddingBikes['latitude'][i], ReddingBikes['longitude'][i], radius, YelpKey, categories, limit).json()
#     YelpVenues.append(venues)  

### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [77]:
# Function to return list of desired results from each yelp venue
def ParseYelpResults(venue): 
    """ 
    Parses elements from a venue in the list of results given by a Yelp Query.

    Args: 
        venue (dict): A venue from the list of Foursquare Venues for each station

    Returns a list of preset elements in the following order from the venue.  
    List order: ['YelpID','venueName','displayAddress','categoryTitle','distance','rating','reviewCount','price']
    """
    # Yelp core data. All venues have this information 
    yelpID = venue['id']
    venueName = venue['name']
    displayAddress = venue['location']['display_address']
    categoryTitle = venue['categories'][0]['alias']
    distance = venue['distance']
    rating = venue['rating']
    reviewCount = venue['review_count']
    
    
    # FS Rich data. Fields which may not exist
    try: 
        price = len(venue['price'])
    except:
        price = None

    returnList = [yelpID, venueName, displayAddress, categoryTitle, distance, rating, reviewCount, price]

    return returnList

In [78]:
# Parsing all results into a list using function for tidy data that can be converted to DF
YelpVenueList = []

for i in range(len(YelpVenues)):
    for venue in YelpVenues[i]['businesses']:
        returnList = ParseYelpResults(venue)
        returnList.extend([i, YelpVenues[i]['total']])
        YelpVenueList.append(returnList)

### Put your parsed results into a DataFrame

In [79]:
yelpColumnsHeader = ['YelpID','venueName', 'displayAddress','categoryTitle','distance','rating','reviewCount','price','stationIndex','totalVenues']

yelpVenuesDF = pd.DataFrame(YelpVenueList)
yelpVenuesDF.columns=yelpColumnsHeader

In [80]:
yelpVenuesDF.head()

,YelpID,venueName,displayAddress,categoryTitle,distance,rating,reviewCount,price,stationIndex,totalVenues
0,rgvwGrdnSPGdw92xypeHmg,Roots Juice Bar,"[1950 Pine St, Redding, CA 96001]",juicebars,65.810927,4.4,62,NaN,0,102
1,bbFV2MAL2KdXZ3txwaI_wQ,The Grape Escape,"[1824 Pine St, Redding, CA 96001]",beer_and_wine,142.235695,4.7,20,2.0,0,102
2,rmkMkpGpEE4sDgdiOYe2BA,TRUE Ride,"[2100 Market St, Redding, CA 96001]",cyclingclasses,164.615711,4.7,29,NaN,0,102
3,-bURSXDTgRK0wFp0_oNbpg,Evergreen - Coffee Shop & Boutique Hotel,"[2085 Pine St, Redding, CA 96001]",coffee,185.533435,4.2,200,2.0,0,102
4,nd7wlihGZ5hm01aaqJhHPg,Market Street Blade And Barrel,"[1777 Market St, Redding, CA 96001]",steak,189.259600,3.6,241,3.0,0,102


In [81]:
# yelpVenuesDF.to_csv('../data/yelpVenuesDF.csv', sep=',')

# Comparing Results

### Which API provided you with more complete data? Provide an explanation. 

Yelp provided much more locations/venues given very similar categories to search for (Due to API limitating 50 calls per location, . Distance from each station was used to determine which ones were called and which ones were exclude for this limitation). Yelp has much more reviews than foursquare. Foursqaure returned some information that were not included from Yelp such as number of photos. Foursquare provided popularity field (score from 0 to 1 of foursquare foot traffic over the last 6 months). Both foursqaure and yelp have reports for buisness or advertisements in-depth metric reports, but from the much larger number of reviews on yelp, it is likely there is much more data on yelp and therefore more insights to be gathered.

[Foursquare Fields](https://docs.foursquare.com/developer/reference/response-fields)

[Yelp Metric Reports](https://docs.developer.yelp.com/reference/create_daily_reports_v3)

In [28]:
print(f'Total number of Foursqaure locations/venues found: {len(FSVenuesDF)}')

# Loop to find total locations/venues found required for Yelp as not all of them returned from API and inserted to dataframe. 
totalYelpLocations = 0

for Venues in YelpVenues:
    totalYelpLocations += Venues['total']

print(f'Total number of Yelp locations/venues found: {totalYelpLocations}')


Total number of Foursqaure locations/venues found: 837
Total number of Yelp locations/venues found: 1859


In [34]:
print(f"Foursqaure average of number of reviews: {FSVenuesDF['totalRatings'].mean()}")
print(f"Yelp average of number of reviews: {yelpVenuesDF['reviewCount'].mean()}")

Foursqaure average of number of reviews: 14.8
Foursqaure average of number of reviews: 71.12158341187559


In [35]:
print(f"Foursqaure Unique Locations: {FSVenuesDF['venueName'].nunique()}")
print(f"Yelp Unique Locations: {yelpVenuesDF['venueName'].nunique()}")

Foursqaure Unique Locations: 133
Foursqaure Unique Locations: 141


### Get the top 10 restaurants according to their rating

In [87]:
print('Top 10 Restaurants from Foursquare:')
FSRestaurantsDF = FSVenuesDF[FSVenuesDF['catIDGroup']=='Dining and Drinking']
FSRestaurantsDF[['venueName','rating','totalRatings']].set_index('venueName').drop_duplicates().sort_values('rating',ascending=False).head(10)


Top 10 Restaurants from Foursquare:


,rating,totalRatings
venueName,,
Theory Collaborative,8.9,35.0
Market Street Steakhouse,8.7,14.0
Club 501,8.3,12.0
Dutch Bros Coffee,8.1,33.0
Anthony's Mediterranean Cuisine,8.1,16.0
Taste & See Creamery,8.0,9.0
Dutch Bros Coffee,7.8,9.0
Wienerschnitzel,7.6,12.0
Dutch Bros Coffee,7.2,10.0


In [89]:
print('Top 10 Restaurants from Yelp:')
yelpVenuesDF['categoryTitle'].unique()
# MultiReviews = yelpVenuesDF[yelpVenuesDF['reviewCount'] > 4]
# MultiReviews[['venueName','rating','reviewCount']].set_index('venueName').drop_duplicates().sort_values('rating',ascending=False).head(10)


Top 10 Restaurants from Yelp:


array(['juicebars', 'beer_and_wine', 'cyclingclasses', 'coffee', 'steak',
       'musicvenues', 'tradamerican', 'salad', 'dancestudio', 'beerbar',
       'theater', 'gymnastics', 'breweries', 'bars', 'meats',
       'artclasses', 'healthtrainers', 'farmersmarket', 'drugstores',
       'catering', 'videogamestores', 'herbsandspices', 'wine_bars',
       'galleries', 'grocery', 'escapegames', 'museums',
       'winetastingroom', 'bootcamps', 'icecream', 'bakeries',
       'festivals', 'sportgoods', 'cocktailbars', 'stadiumsarenas',
       'seafood', 'donuts', 'paintandsip', 'recreation',
       'musicalinstrumentsandteachers', 'tours', 'boatcharters',
       'breakfast_brunch', 'gaybars', 'social_clubs', 'newamerican',
       'diveshops', 'cosmetics', 'parks', 'golf', 'swimmingpools',
       'playgrounds', 'fooddeliveryservices', 'bubbletea', 'gyms',
       'servicestations', 'convenience', 'rafting', 'bikes', 'foodtrucks',
       'paddleboarding', 'boating', 'horsebackriding', 'fishing'

In [92]:
YelpVenues[0]['businesses'][10]

{'id': 'CKUtFK5sqlRYxQN_FPDghg',
 'alias': 'riverfront-playhouse-redding',
 'name': 'Riverfront Playhouse',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/g-tkk4mzBB2VFJrsw8BIHA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/riverfront-playhouse-redding?adjust_creative=aOiDJz5PxNb73roKFPGFQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=aOiDJz5PxNb73roKFPGFQg',
 'review_count': 4,
 'categories': [{'alias': 'theater', 'title': 'Performing Arts'}],
 'rating': 4.8,
 'coordinates': {'latitude': 40.57975126019169,
  'longitude': -122.39174263809814},
 'transactions': [],
 'location': {'address1': '1950 California St',
  'address2': None,
  'address3': None,
  'city': 'Redding',
  'zip_code': '96002',
  'country': 'US',
  'state': 'CA',
  'display_address': ['1950 California St', 'Redding, CA 96002']},
 'phone': '+15302211028',
 'display_phone': '(530) 221-1028',
 'distance': 241.92033741688513,
 'attributes': {'business_temp_closed': None, 'wa

In [93]:
headers = {
    'Authorization': f'Bearer {YelpKey}',
}
url = 'https://api.yelp.com/v3/businesses/search'
params = {
    'location': 'New York, NY',
    'categories': 'restaurants'  # Example of multiple categories
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

In [98]:
data['businesses'][3]

{'id': 'nRO136GRieGtxz18uD61DA',
 'alias': 'eleven-madison-park-new-york',
 'name': 'Eleven Madison Park',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/s_H7gm_Hwmz--O6bo1iU-A/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/eleven-madison-park-new-york?adjust_creative=aOiDJz5PxNb73roKFPGFQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=aOiDJz5PxNb73roKFPGFQg',
 'review_count': 2495,
 'categories': [{'alias': 'newamerican', 'title': 'New American'},
  {'alias': 'french', 'title': 'French'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.3,
 'coordinates': {'latitude': 40.7416907417333, 'longitude': -73.9872074872255},
 'transactions': [],
 'price': '$$$$',
 'location': {'address1': '11 Madison Ave',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10010',
  'country': 'US',
  'state': 'NY',
  'display_address': ['11 Madison Ave', 'New York, NY 10010']},
 'phone': '+12128890905',
 'display_phone